In [3]:
import pandas as pd
import re
import os
import sqlite3
import numpy as np

df = pd.read_csv('불러올 csv파일 이름', names=['order','news_name','pubdate','url', 'content'])   # 컬럼이름 지정
df.to_csv('저장할 csv파일 이름', encoding='utf-8-sig')

con = sqlite3.connect("db.news_test02")   # 데이터베이스 이름
c = con.cursor()

contents_new = []

def preprocessing_articles(data):
    for content in data['content']:

        content = content.split('\n')
        content = list(filter(lambda s: len(s) > 3, content))

        # 글 마지막에 항상 기자의 메일 주소가 추가되어 있다. 그 이후로 나오는 글은 불필요하므로 삭제함
        for i in range(1, len(content)):
            if '@' in content[-i]:
                break               
            if i == len(content)-1:
                i = 1
            content_prep = content[:-i]

        # 너무 짧거나, 광고, copyright, 사진설명, 메일 등 불필요한 단어가 포함되어 있을 경우 불필요한 문장으로 판단하고 삭제 처리
            content_prep = list(filter(lambda s: len(s) > 30 and \
                '▶' not in s and \
                    '©' not in s and \
                        '▲' not in s and \
                            '사진=' not in s and \
                                '사진제공=' not in s and \
                                    '@' not in s and \
                                        not re.findall("기자 *$", s) and \
                                            not re.findall("제공 *$", s) and \
                                                not re.findall("이하", s) and \
                                                    not re.findall("무단전재", s) and \
                                                        not re.findall("기자 *= *", s) , content_prep))

        # 언론사명이 들어간 prefix 삭제
        content_prep = list(map(lambda s: re.sub("\[.*\]", "", s), content_prep)) #[스포츠서울]
        content_prep = list(map(lambda s: re.sub("\(.*\=.*\)", "", s), content_prep)) #(서울=뉴시스)
        content_prep = list(map(lambda s: re.sub("\【.*\=.*\】", "", s), content_prep)) #【파이낸셜뉴스 포천=강근주 기자】
        content_prep = list(map(lambda s: re.sub('[^가-힣]',' ', s), content_prep)) # 한글 제외 영문, 한자, 숫자, 특수문자 모두 제거 

        contents_new.append('\n'.join(content_prep))

    data['news_content'] = contents_new

    return data


if __name__ == "__main__":
        
    file = '위에서 저장한 csv파일 이름'
    data = pd.read_csv(file)

    base_name, ext = os.path.splitext(os.path.basename(file))
    output_file = "{}_{}{}".format(base_name, "prep", ext)   # 파일이름을 위에서 저장한 파일이름에 prep을 붙여 생성

    data_prep = preprocessing_articles(data)
    data_prep['news_content'].replace('', np.nan, inplace=True)    # 빈셀이 있는 행을 삭제하기 위해 NaN으로 변경
    data_prep_drop = data_prep.dropna(axis=0)
    data_prep_drop_dup = data_prep_drop.drop_duplicates(['news_content'], keep='first', inplace=False, ignore_index=True)   # 중복값 제거
    result = data_prep_drop_dup[['news_name', 'pubdate', 'url', 'news_content']]
    result.to_sql('news_dummy', con, if_exists='append')         # DB에 news_dummy 라는 테이블 이름으로 저장, 테이블이 존재하면 새 값을 추가
    con.commit()
    con.close()
    result.to_csv(output_file, index=None, encoding='utf-8-sig')   # 테이터 확인을 위해 csv파일로 저장